In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessedmaritimedata/preprocessedmarinedata.csv
/kaggle/input/preprocessedmaritimedata/shipinitialtime.csv


data = pd.read_csv("/kaggle/input/upmarinedata/updatedMarineData.csv")

In [ ]:
# forumla to convert heading into a unit vector
# (sine(heading), cos(heading))
# then you can multiply by speed for a more accurate picture of 
# what the vector is doing
import numpy as np
heading = 0
def HeadingToVector(heading):
    vectorX = np.sin(heading * np.pi / 180)
    vectorY = np.cos(heading * np.pi / 180)
    # Fix floating point errors
    if(vectorX < 0.00001 and vectorX > -0.00001):
        vectorX = 0
    if(vectorY < 0.00001 and vectorY > -0.00001):
        vectorY = 0
    return (vectorX, vectorY)

HeadingToVector(heading)

In [ ]:
for index, row in data.iterrows():
    data.loc[index, "VelocityX"] = HeadingToVector(row["heading"])[0]
    data.loc[index, "VelocityY"] = HeadingToVector(row["heading"])[1]

In [ ]:
data.to_csv('updatedMarineData.csv',index=False)
data.iloc[1000000:1000100]

In [ ]:
data = pd.read_csv("/kaggle/input/updatedmarinedataset/updatedMarineData.csv")

# PREPROCESSING

In [ ]:
# Get relevant data (data filtering)
data = data[["timestamp", "mmsi", "lat", "lon", "speed", "heading"]]

In [ ]:
import seaborn as sns
# Outliers - Speeds over 50 removed, since they are likely errors due to noise
sns.boxplot(data["speed"])
data = data.loc[data["speed"] <= 50]

In [ ]:
# Remove null values
data.dropna(inplace=True)
data.shape[0]

# Now for path prediction (initially done on one example ship, 205366000, but updated to be done on all ships)

In [ ]:
#testShip = data.loc[(data["mmsi"] == 205366000) | (data["mmsi"] == 205451000)]
testShip = data
testShip.shape[0]

In [ ]:
testShip.head()

In [ ]:
# figure out time passed
from datetime import datetime

testShip["timePassed"] = 0

for i in range(len(testShip["timestamp"])):
    dateObj = datetime.strptime(testShip.iloc[i, testShip.columns.get_loc("timestamp")], '%Y-%m-%d %H:%M:%S')
    testShip.iloc[i, testShip.columns.get_loc("timePassed")] = dateObj.timestamp()

In [ ]:
testShip.head()

In [ ]:
testShip = pd.read_csv("/kaggle/input/temper/temp.csv")

In [ ]:
checkList = pd.DataFrame(columns=["mmsi", "timeStart"])
for i in range(len(testShip["mmsi"])):
    mmsi = testShip.iloc[i, testShip.columns.get_loc("mmsi")]
    timePassed = testShip.iloc[i, testShip.columns.get_loc("timePassed")]
    timeStart = checkList.loc[checkList["mmsi"] == mmsi]
    
    if(timeStart.size == 0): # returned empty, put timeStart in table
        checkList.loc[len(checkList)] = [mmsi, timePassed]
        timeStart = checkList.loc[checkList["mmsi"] == mmsi]

    # update with new time
    testShip.iloc[i, testShip.columns.get_loc("timePassed")] = timePassed - timeStart["timeStart"]
        
checkList.head()

In [ ]:
testShip.loc[testShip["mmsi"] == 205451000].head()

In [ ]:
testShip.head()

In [ ]:
testShip.to_csv('preprocessedmarinedata.csv', index=False)

In [ ]:
checkList.to_csv('shipinitialtime.csv', index=False)

In [ ]:
data = pd.read_csv("/kaggle/input/preprocessedmarine/preprocessedmarinedata.csv")
iniTime = pd.read_csv("/kaggle/input/preprocessedmarine/shipinitialtime.csv")

In [ ]:
# Also get time step for each ship

In [ ]:
data.head()

In [ ]:
checkList = pd.DataFrame(columns=["mmsi", "lastStep"])
data["timeStep"] = 0

In [ ]:
start = 0
end = 1000000
finish = len(data["mmsi"])
print(data.shape[0])

In [ ]:
for i in range(end):
    i += start
    if(i >= finish):
        print("finish")
        start -= end
        break
    
    mmsi = data.iloc[i, data.columns.get_loc("mmsi")]
    timePassed = data.iloc[i, data.columns.get_loc("timePassed")]
    timeStart = checkList.loc[checkList["mmsi"] == mmsi]
    
    if(timeStart.size == 0): # returned empty, put 0 in table
        checkList.loc[len(checkList)] = [mmsi, 0]
        timeStart = checkList.loc[checkList["mmsi"] == mmsi]

    # update with new time
    data.iloc[i, data.columns.get_loc("timeStep")] = timePassed - timeStart["lastStep"]
    checkList.loc[checkList["mmsi"] == mmsi, "lastStep"] = timePassed
        
start += end
print(start)

In [ ]:
data.to_csv("preprocessedmarinedata2.csv", index=False)

In [ ]:
data.iloc[13828472:13828475].head()
data.head()
# note- one hot encoding? 4 hot encoding?

In [7]:
data = pd.read_csv("/kaggle/input/preprocessedmaritimedata/preprocessedmarinedata.csv")
iniTime = pd.read_csv("/kaggle/input/preprocessedmaritimedata/shipinitialtime.csv")

# LSTM

In [37]:
# Split data accordingly
from sklearn.model_selection import train_test_split

X = data[["lat", "lon", "speed", "heading"]]
y = data[["timestamp", "mmsi", "timePassed", "timeStep"]]
X_train = X.iloc[:10000]
y_train = y.iloc[:10000]
X_test = X.iloc[10000:20000]
y_test = y.iloc[10000:20000]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.9, random_state=42)


In [38]:
# Do necessary scaling and transforming of the data to fit the model
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (10000, 1, 4)
X_test shape: (10000, 1, 4)


In [39]:
# Import the model and functions needed
from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [40]:
# defining the LSTM, it's layers, inputs, outputs, ect.
def autoencoder_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(4, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(4, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)
    model = Model(inputs=inputs, outputs=output)
    return model

In [41]:
# create model
model = autoencoder_model(X_train)
model.compile(optimizer='adam', loss='mae')
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 1, 16)          │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 4)              │           336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_2 (RepeatVector)  │ (None, 1, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 1, 4)           │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 1, 16)          │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 1, 4)           │            68 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,236 (12.64 KB)

 Trainable params: 3,236 (12.64 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# run tests by fitting data to model
epochs = 100
batch = 10
history = model.fit(X_train, X_train, epochs=epochs, batch_size=batch, validation_split=.2, verbose=0).history

In [43]:
import plotly.graph_objects as go
# Show off loss function
fig = go.Figure()

fig.add_trace(go.Scatter(x=[x for x in range(len(history['loss']))], y=history['loss'],
                    mode='lines',
                    name='loss'))

fig.add_trace(go.Scatter(x=[x for x in range(len(history['val_loss']))], y=history['val_loss'],
                    mode='lines',
                    name='validation loss'))

fig.update_layout(title="Autoencoder error loss over epochs",
                  yaxis=dict(title="Loss"),
                  xaxis=dict(title="Epoch"))

fig.show()

In [44]:
# Create predictions and reformat into normal form
X_pred = model.predict(X_train)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = scaler.inverse_transform(X_pred)
X_pred = pd.DataFrame(X_pred, columns= X.columns)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


In [46]:
X_train

array([[[0.02826052, 0.01534304, 0.18293796, 0.15598886]],

       [[0.03023589, 0.01724069, 0.19023723, 0.22562674]],

       [[0.03011908, 0.0185651 , 0.20437956, 0.25626741]],

       ...,

       [[0.10911319, 0.41276511, 0.22764599, 0.72423398]],

       [[0.10864045, 0.4112067 , 0.22536496, 0.72423398]],

       [[0.10770677, 0.40805022, 0.22764599, 0.72423398]]])

In [48]:
# Find differences between real data set and predicted data set
scores = pd.DataFrame()
scores['lat_train'] = X.loc[:, "lat"].iloc[:10000]#X_train['lat']
scores["lat_predicted"] = X_pred["lat"]
scores['loss_mae'] = (scores['lat_train']-scores['lat_predicted']).abs()

In [49]:
# Display aforementioned differences
fig = go.Figure(data=[go.Histogram(x=scores['loss_mae'])])
fig.update_layout(title="Error distribution", 
                 xaxis=dict(title="Error delta between predicted and real data [lat]"),
                 yaxis=dict(title="Data point counts"))
fig.show()

In [54]:
# make predictions on test data
X_pred = model.predict(X_test)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = scaler.inverse_transform(X_pred)
X_pred = pd.DataFrame(X_pred, columns=X.columns)
X_pred.index = X.iloc[10000:20000].index

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [58]:
# Update Scores, Determine Anomaly threshold
scores = X_pred
scores['datetime'] = y_test["timestamp"]
scores['real lat'] = X.loc[:, "lat"].iloc[10000:20000]#X_test["lat"]
scores["loss_mae"] = (scores['real lat'] - scores['lat']).abs()
scores['Threshold'] = 0.5
scores['Anomaly'] = np.where(scores["loss_mae"] > scores["Threshold"], 1, 0)

In [59]:
# Show Anomaly Representation figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=scores['datetime'], 
                         y=scores['loss_mae'], 
                         name="Loss"))
fig.add_trace(go.Scatter(x=scores['datetime'], 
                         y=scores['Threshold'],
                         name="Threshold"))

fig.update_layout(title="Error Timeseries and Threshold", 
                 xaxis=dict(title="DateTime"),
                 yaxis=dict(title="Loss"))
fig.show()

In [60]:
# Number of Anomalies
scores['Anomaly'].value_counts()

Anomaly
0    9673
1     327
Name: count, dtype: int64

In [61]:
# Put anomalies inside scores
anomalies = scores[scores['Anomaly'] == 1][['real lat']]
anomalies = anomalies.rename(columns={'real lat':'anomalies'})
scores = scores.merge(anomalies, left_index=True, right_index=True, how='left')

In [62]:
# Showcase Anomalies in Red
fig = go.Figure()

fig.add_trace(go.Scatter(x=scores["datetime"], y=scores["real lat"],
                    mode='lines',
                    name='Latitude'))

fig.add_trace(go.Scatter(x=scores["datetime"], y=scores["anomalies"],
                    name='Anomaly', 
                    mode='markers',
                    marker=dict(color="red",
                                size=11,
                                line=dict(color="red",
                                          width=2))))

fig.update_layout(title_text="Anomalies Detected LSTM Autoencoder")

fig.show()